# NBADataEngineering
Capstone project for the Udacity Data Engineering Nanodegree. In this project, we perform an ETL process on a set of NBA datasets.  

As a huge NBA fan, I'd like to extract and organise some data and insights from my favourite teams
and players. I will reorder the csvs in such a way that I can easily query for augmented information that combines the 
data from the original sources. By doing this, the resulting database will be an excellent example of an Analytics table.
The data will be extracted from 5 different csvs: 
- **historical_nba_performance.csv**  
This dataset contains information about team performance per year.
- **nba_all_star_games.csv**  
This dataset includes information about the players participating in the yearly All-Star games.
- **nba_shots_2000_to_2018.csv**
This dataset (the biggest one with over 1m rows) shows a detailed description of all the shots made from 2000 to 2018 on every game.
- **player_data.csv**  
Information about each player.
- **players.csv**
More informacion about players.  

### Installation
First things first. To correctly run the project, please install the dependencies on an empty python environment.
```
# Run this on the project's source folder. I am using conda in this example.
conda create --name dec python=3.7 --no-default-packages
pip install -e .
```


### Exploratory Data Analysis

In [1]:
import pandas as pd

# Setting the chunksize to 100 since we just want to take a first look into the data ;) 
historical = pd.read_csv("data/historical_nba_performance.csv", chunksize=100).get_chunk()
all_star = pd.read_csv("data/all_star.csv", chunksize=100).get_chunk()
shots = pd.read_csv("data/NBA_Shots_2000_to_2018.csv", chunksize=100).get_chunk()
player_data_1 = pd.read_csv("data/player_data.csv", chunksize=100).get_chunk()
player_data_2 = pd.read_csv("data/Players.csv", chunksize=100).get_chunk()

#### Historical Teams table

In [2]:
historical.head()

,year,team,record,winning_percentage
0,2016-17,Celtics,25-15,0.625
1,2015-16,Celtics,48-34,0.585
2,2014-15,Celtics,40-42,0.488
3,2013-14,Celtics,25-57,0.305
4,2012-13,Celtics,41-40,0.506


Ok, so apparently this csv has 4 columns and a number of unnamed columns that were probably reserved for extra values. The columns are:
- Year
- Team
- Record
- Winning Percentage

#### All-Star teams table

In [3]:
all_star.head()

,year,player,pos,ht,wt,team,selection_type,nba_draft_status,nationality
0,2016.0,Stephen Curry,G,6-3,190.0,Golden State Warriors,Western All-Star Fan Vote Selection,2009 Rnd 1 Pick 7,United States
1,2016.0,James Harden,SG,6-5,220.0,Houston Rockets,Western All-Star Fan Vote Selection,2009 Rnd 1 Pick 3,United States
2,2016.0,Kevin Durant,SF,6-9,240.0,Golden State Warriors,Western All-Star Fan Vote Selection,2007 Rnd 1 Pick 2,United States
3,2016.0,Kawhi Leonard,F,6-7,230.0,San Antonio Spurs,Western All-Star Fan Vote Selection,2011 Rnd 1 Pick 15,United States
4,2016.0,Anthony Davis,PF,6-11,253.0,New Orleans Pelicans,Western All-Star Fan Vote Selection,2012 Rnd 1 Pick 1,United States


In [4]:
print([col for col in all_star.columns if 'Unnamed' not in col ])

['year', 'player', 'pos', 'ht', 'wt', 'team', 'selection_type', 'nba_draft_status', 'nationality']


#### Nba Shots

In [5]:
shots.head()

,x,id,player,season,top_px_location,leftpx_location,date,team,opponent,location,quarter,game_clock,outcome_1_if_made_0_otherwise,shotvalue,shotdistance_ft,team_score,opponentscore
0,1,abdulma02,Mahmoud Abdul-Rauf,2001,250,304.0,110600.0,VAN,ATL,HOME,3,00:38.4,0.0,2.0,21.0,69.0,55.0
1,2,abdulma02,Mahmoud Abdul-Rauf,2001,147,241.0,111800.0,VAN,DAL,HOME,2,9:22,1.0,2.0,10.0,33.0,26.0
2,3,abdulma02,Mahmoud Abdul-Rauf,2001,132,403.0,112400.0,VAN,DET,AWAY,3,6:42,0.0,2.0,18.0,60.0,78.0
3,4,abdulma02,Mahmoud Abdul-Rauf,2001,177,129.0,112400.0,VAN,DET,AWAY,3,2:42,0.0,2.0,17.0,66.0,80.0
4,5,abdulma02,Mahmoud Abdul-Rauf,2001,99,390.0,112400.0,VAN,DET,AWAY,3,2:18,0.0,2.0,16.0,66.0,80.0


In [6]:
print([col for col in shots.columns if 'Unnamed' not in col ])

['x', 'id', 'player', 'season', 'top_px_location', 'leftpx_location', 'date', 'team', 'opponent', 'location', 'quarter', 'game_clock', 'outcome_1_if_made_0_otherwise', 'shotvalue', 'shotdistance_ft', 'team_score', 'opponentscore']


The info on this table makes up for a great facts table.

#### players_data and Players table

In [7]:
player_data_1.head()

,name,year_start,year_end,position,height,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240,"June 24, 1968",Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235,"April 7, 1946",Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225,"April 16, 1947","University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162,"March 9, 1969",Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223,"November 3, 1974",San Jose State University


In [8]:
player_data_2.head()

,player,height,weight,collage,born,birth_city,birth_state
0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


In [9]:
print([col for col in player_data_1.columns if 'Unnamed' not in col ])
print([col for col in player_data_2.columns if 'Unnamed' not in col ])

['name', 'year_start', 'year_end', 'position', 'height', 'weight', 'birth_date', 'college']
['player', 'height', 'weight', 'collage', 'born', 'birth_city', 'birth_state']


As you can notice, both tables contain similar data, so in the architecture we will make sure to combine this tables with their useful info.

### Proposed Data Model

Given the nature of the data, I am going to use a Star schema to create a RedshiftDB with 4 nodes. The idea is to host the player dimensions and the team dimensions on each node (distribution type ALL), and the shots fact table will be distributed across all 4 nodes. This will allow us to expand the database horizontally if needed, and also it will allow us to access the data faster since each node will contain the necessary information.

![](data/images/star.png)

Note that the players table will extrapolate some info from the "all_star" original csv, in particular the nba_draft_status and the nationality of the player.  

This architecture will let us extract and augment useful information regarding the shots made.
Possible queries to answer:

- Teams with the most cumulative points scored across seasons.
- Players with the most cumulative points scored.
- Players with the best 3 point scoring percentage.
- Extract the colleges with the most cumulative player scores.

### Proposed Pipeline

![](data/images/nba.png)

### Interesting Queries

#### Teams with the most cumulative points across seasons.

In [1]:
"""
SELECT team, season, SUM(shot_value) AS cumulative_points
FROM fact_shots 
WHERE
    converted = TRUE
GROUP BY(team, season)
ORDER BY cumulative_points DESC;
"""

UsageError: Line magic function `%sql` not found.


#### Players with most cumulative points scored

In [2]:
"""
SELECT player_id, SUM(shot_value) AS cumulative_points
FROM fact_shots 
WHERE
    converted = TRUE
GROUP BY(player_id)
ORDER BY cumulative_points DESC;
"""

'\nSELECT player_id, SUM(shot_value) AS cumulative_points\nFROM fact_shots \nWHERE\n    converted = TRUE\nGROUP BY(player_id)\nORDER BY cumulative_points DESC;\n'

#### Players with best 3 point score percentage

In [4]:
"""
WITH total_3 AS (
    SELECT player_id, SUM(shot_value) AS t3 
    FROM fact_shots 
    WHERE shot_value = 3 
    GROUP BY(player_id) 
    ORDER BY t3 DESC
), made_3 AS (
    SELECT player_id, SUM(shot_value) AS m3 
    FROM fact_shots 
    WHERE shot_value = 3 AND converted = TRUE 
    GROUP BY(player_id) 
    ORDER BY m3 DESC
)
SELECT made_3.player_id, made_3.m3, total_3.t3, (made_3.m3 / total_3.t3) AS percentage 
FROM total_3, made_3 WHERE made_3.player_id = total_3.player_id
ORDER BY percentage DESC;
"""

'\nWITH total_3 AS (\n    SELECT player_id, SUM(shot_value) AS t3 \n    FROM fact_shots \n    WHERE shot_value = 3 \n    GROUP BY(player_id) \n    ORDER BY t3 DESC\n), made_3 AS (\n    SELECT player_id, SUM(shot_value) AS m3 \n    FROM fact_shots \n    WHERE shot_value = 3 AND converted = TRUE \n    GROUP BY(player_id) \n    ORDER BY m3 DESC\n)\nSELECT made_3.player_id, made_3.m3, total_3.t3, (made_3.m3 / total_3.t3) AS percentage \nFROM total_3, made_3 WHERE made_3.player_id = total_3.player_id\nORDER BY percentage DESC;\n'

#### Colleges with the most cumulative player scores

In [5]:
"""
WITH made AS (
    SELECT player_id, SUM(shot_value) AS m 
    FROM fact_shots 
    WHERE converted = TRUE 
    GROUP BY(player_id) 
    ORDER BY m DESC
)
SELECT college, SUM(total_points_made) AS total_points_made_by_college 
FROM
    (SELECT dp.player_name, dp.college, made.player_id, made.m AS total_points_made
    FROM made JOIN dimension_player dp ON made.player_id = dp.player_id)
GROUP BY college
ORDER BY total_points_made_by_college DESC;
"""

'\nWITH made AS (\n    SELECT player_id, SUM(shot_value) AS m \n    FROM fact_shots \n    WHERE converted = TRUE \n    GROUP BY(player_id) \n    ORDER BY m DESC\n)\nSELECT college, SUM(total_points_made) AS total_points_made_by_college \nFROM\n    (SELECT dp.player_name, dp.college, made.player_id, made.m AS total_points_made\n    FROM made JOIN dimension_player dp ON made.player_id = dp.player_id)\nGROUP BY college\nORDER BY total_points_made_by_college DESC;\n'